In [ ]:
from vllm import LLM, SamplingParams
import torch 
import pandas as pd
from torchmetrics.text import ROUGEScore


In [ ]:
rouge = ROUGEScore()

In [ ]:
sampling_params = SamplingParams(temperature=0.0, top_p=0.95, max_tokens=512)

In [ ]:
batch_size=64

In [ ]:
rev_df = pd.read_csv("../../../data/labelled/reviews/splits/test.csv")

In [ ]:
llm = LLM(model="llama_model", dtype=torch.float16, gpu_memory_utilization=0.95, enforce_eager=True)

INFO 04-14 11:40:40 llm_engine.py:75] Initializing an LLM engine (v0.4.0) with config: model='llama_model', tokenizer='llama_model', tokenizer_mode=auto, revision=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.float16, max_seq_len=4096, download_dir=None, load_format=auto, tensor_parallel_size=1, disable_custom_all_reduce=True, quantization=None, enforce_eager=True, kv_cache_dtype=auto, device_config=cuda, seed=0)


You set `add_prefix_space`. The tokenizer needs to be converted from the slow tokenizers


INFO 04-14 11:40:40 selector.py:45] Cannot use FlashAttention because the package is not found. Please install it for better performance.
INFO 04-14 11:40:40 selector.py:21] Using XFormers backend.
INFO 04-14 11:40:45 model_runner.py:104] Loading model weights took 12.5523 GB
INFO 04-14 11:40:46 gpu_executor.py:94] # GPU blocks: 7934, # CPU blocks: 512


In [ ]:
preds = []
for idx in range(0, len(rev_df), batch_size):
    batch = rev_df[idx:idx+batch_size]
    descs = batch.input_reviews.to_list()
    labels = batch.label_reviews.to_list()
    input_batch = ["<s>[INST] "+desc+" [/INST]" for desc in descs]
    outputs = llm.generate(input_batch, sampling_params, use_tqdm=False)
    preds.extend([out.outputs[0].text.strip() for out in outputs])    

In [ ]:
rev_df['llama'] = preds

In [ ]:
rev_df.to_csv("llama_results_reviews.csv", index=False)

In [ ]:
rev_df.head(2)

,input_reviews,label_reviews,llama
0,Below are the reviews of a product: \n- I purc...,pros:\n- Reduces glare\n- Very impressive\n- W...,"Based on the reviews provided, here are the pr..."
1,Below are the reviews of a product: \n- Well w...,pros:\n- Huge capacity\n- Beautiful design\n- ...,"Based on the reviews provided, here are the pr..."


In [ ]:
rouge_scores = {"rouge1":[], "rouge2":[], "rougeL": []}
for pred, target in zip(rev_df.label_reviews.to_list(), rev_df.llama.to_list()):
    if str(target).lower() == "nan":
        target = ""
    rouge_all = rouge(pred, target)
    rouge_scores['rouge1'].append(rouge_all['rouge1_fmeasure'].item())
    rouge_scores['rouge2'].append(rouge_all['rouge2_fmeasure'].item())
    rouge_scores['rougeL'].append(rouge_all['rougeL_fmeasure'].item())

In [ ]:
rev_df['rouge1'] = rouge_scores['rouge1']
rev_df['rouge2'] = rouge_scores['rouge2']
rev_df['rougeL'] = rouge_scores['rougeL']

In [ ]:
rev_df.to_csv("llama_results_reviews.csv", index=False)

In [ ]:
scores = {"rouge1": rev_df.rouge1.mean(), "rouge2": rev_df.rouge2.mean(), "rougeL": rev_df.rougeL.mean()}

In [ ]:
scores

{'rouge1': 0.3298645227672532,
 'rouge2': 0.1131155245625414,
 'rougeL': 0.20740494322949088}